In [ ]:
import requests
import json
from datetime import datetime

# -------------------- CONFIG --------------------
WEATHER_API_URL = "https://api.open-meteo.com/v1/forecast"
LAT, LON = 33.9167, 130.7667  # Tokyo coordinates
AQI_API = "https://air-quality-api.open-meteo.com/v1/air-quality"

def get_weather_json():
    try:
        # Weather API request
        params = {
            "latitude": LAT,
            "longitude": LON,
            "current_weather": True,
            "hourly": "uv_index,precipitation,relative_humidity_2m,windspeed_10m,temperature_2m",
            "timezone": "auto"
        }
        weather_res = requests.get(WEATHER_API_URL, params=params, timeout=5)
        weather_res.raise_for_status()
        weather_data = weather_res.json()

        # AQI API request
        aqi_res = requests.get(AQI_API, params={
            "latitude": LAT,
            "longitude": LON,
            "hourly": "european_aqi",
            "timezone": "auto"
        }, timeout=5)
        aqi_res.raise_for_status()
        aqi_data = aqi_res.json()

        # Combine both responses
        combined_response = {
            "weather_data": weather_data,
            "aqi_data": aqi_data
        }

        # Pretty-print the JSON response
        print("\n📊 Raw Weather API JSON Response:")
        print(json.dumps(combined_response, indent=2))

        return combined_response

    except Exception as e:
        print(f"⚠️ Weather API fetch failed: {e}")
        return None

def main():
    print(f"Fetching weather data for coordinates: ({LAT}, {LON})")
    print(f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    get_weather_json()

if __name__ == "__main__":
    main()

In [14]:
def get_weather_json():
    try:
        # Weather API request
        params = {
            "latitude": LAT,
            "longitude": LON,
            "current_weather": True,
            "hourly": "uv_index,precipitation,relative_humidity_2m,windspeed_10m,temperature_2m",
            "timezone": "auto"
        }
        weather_res = requests.get(WEATHER_API_URL, params=params, timeout=5)
        weather_res.raise_for_status()
        weather_data = weather_res.json()

        # AQI API request
        aqi_res = requests.get(AQI_API, params={
            "latitude": LAT,
            "longitude": LON,
            "hourly": "european_aqi",
            "timezone": "auto"
        }, timeout=5)
        aqi_res.raise_for_status()
        aqi_data = aqi_res.json()

        # Find common timestamps
        weather_times = weather_data["hourly"]["time"]
        aqi_times = aqi_data["hourly"]["time"]
        common_times = list(set(weather_times) & set(aqi_times))
        if not common_times:
            raise Exception("No common timestamps between weather and AQI data.")

        # Use the latest common timestamp
        latest_common_time = max(common_times)
        index = weather_times.index(latest_common_time)

        # Extract current values
        temperature = weather_data["hourly"]["temperature_2m"][index]
        windspeed = weather_data["hourly"]["windspeed_10m"][index]
        humidity = weather_data["hourly"]["relative_humidity_2m"][index]
        uv_index = weather_data["hourly"]["uv_index"][index]
        precipitation = weather_data["hourly"]["precipitation"][index]
        aqi_index = aqi_data["hourly"]["time"].index(latest_common_time)
        aqi = aqi_data["hourly"]["european_aqi"][aqi_index]

        # Print current conditions
        print("\n🌤️ Current Weather Conditions:")
        print(f"Time: {latest_common_time}")
        print(f"Temperature: {temperature} °C")
        print(f"Humidity: {humidity} %")
        print(f"UV Index: {uv_index}")
        print(f"Windspeed: {windspeed} km/h")
        print(f"Precipitation: {precipitation} mm")
        print(f"Air Quality Index (AQI): {aqi}")

    except Exception as e:
        print(f"⚠️ Weather API fetch failed: {e}")


In [15]:
get_weather_json()


🌤️ Current Weather Conditions:
Time: 2025-08-05T23:00
Temperature: 28.6 °C
Humidity: 76 %
UV Index: 0.0
Windspeed: 13.2 km/h
Precipitation: 0.0 mm
Air Quality Index (AQI): None
